In [2]:
import pandas as pd
import os
from globals import glob
from utilities import prepare_database
import configparser
import sys
sys.path.append("../tools/")
import _db_tools as db #Personal tool for managing sqlite databases in data science

####Helper Functions####
def mean_calculator(measures: pd.DataFrame, lenses_per_nest=None) -> list[float]:
    '''Calculate the desired means:
    Parameters:
    - measures (DataFrame): The input DataFrame containing fiber measurements.
    - lenses_per_nest (int, optional): The number of lenses per nest for specific means calculation. If None, global means are calculated.
    - Returns:
    list: A list containing the mean values for fbx and fby. If lenses_per_nest is specified, it returns specific means for each position.
    If lenses_per_nest is None:
    - Calculates a global mean for fbx and fby.
    - Returns the mean values for both fbx and fby in a list.
    - Displays the mean values for fbx and fby.
    If lenses_per_nest is specified:
    - Calculates specific means for each position for fbx and fby based on the number of lenses per nest.
    - Returns a list containing specific mean values for fbx and fby for each position.
    - Displays the specific mean values for fbx and fby per position.'''
    resume = measures.transpose().describe() #Transpose the df first due to describe() working in columns.
    rough_means = list(resume.iloc[1, :].values)
    means = []; means_fbx = []; means_fby = [] #Preallocation
    if lenses_per_nest == None: #Calculates a global mean for fbx and for fby
        for i, mean in enumerate(rough_means): #Iterates and rounds every mean value
            mean = round(mean, 4)
            means_fbx.append(mean) if i % 2 == 0 else means_fby.append(mean)
            means.append(mean)
        abs_mean_fbx = sum(means_fbx) / len(means_fbx)
        abs_mean_fby = sum(means_fby) / len(means_fby)
        means = [abs_mean_fbx, abs_mean_fby]
        print("Means (fbx and fby):") 
        print("Fiber x: " + str(round(abs_mean_fbx, 4)))
        print("Fiber y: " + str(round(abs_mean_fby, 4)))
    else: #Calculates specific means for each position for fbx and fby
        for index in range(lenses_per_nest*2):
            if index % 2 == 0:
                mean_fbx = rough_means[0::2] #Gets fbx values
                mean_fbx = mean_fbx[index::lenses_per_nest] #Gets the values of the specific lens
                abs_mean_fbx = sum(mean_fbx) / len(mean_fbx)
                means_fbx.append(abs_mean_fbx)
            else:
                mean_fby = rough_means[0::2] #Gets fby values
                mean_fby = mean_fby[index::lenses_per_nest] #Gets the values of the specific lens
                abs_mean_fby = sum(mean_fby) / len(mean_fby)
                means_fby.append(abs_mean_fby)
        means = means_fbx + means_fby
        print("Means per position (from lower to higher):") 
        print("  Fiber x: ")
        print([round(value, 4) for value in means_fbx])
        print("  Fiber y: ")
        print([round(value, 4) for value in means_fby])
    return means

def limits_gen(measurements: pd.DataFrame, means: list, lenses_per_nest = None) -> pd.DataFrame:
    '''Generate the limit values for a list containing the means in a DataFrame.
    Calculates the total mean for each fiber axis and applies it to the corresponding rows.
    Parameters:
    - measures (pd.DataFrame): The measurements dataframe to get its size.
    - means (list): A list of means to generate limits for.
    - lenses_per_nest (int, optional): The number of lenses per nest for specific means calculation. If None, global means are calculated.
    Returns:
    - limits: A dataframe containing the generated limits.'''
    x_tolerance = glob.x_tolerance
    y_tolerance = glob.y_tolerance
    limits = pd.DataFrame(columns=["LO_LIMIT", "HI_LIMIT"]) #Columns names
    if lenses_per_nest == None: #Calculates a global mean for fbx and for fby
        for index in range(int(measurements.shape[0] / (lenses_per_nest * 2))): #Iterates over the positions of the dataframe
            if index % 2 == 0: #Fbx rows
                low_limit = round(means[0] - x_tolerance, 4)
                high_limit = round(means[0] + x_tolerance, 4)
            else: #Fby rows
                low_limit = round(means[1] - y_tolerance, 4)
                high_limit = round(means[1] + y_tolerance, 4)
            current_limits_df = pd.DataFrame({"LO_LIMIT": [low_limit], "HI_LIMIT": [high_limit]}) #Create a DataFrame with the current low_limit and high_limit values
            limits = pd.concat([limits, current_limits_df], ignore_index=True, axis=0) #Concatenate the current limits DataFrame with the main 'limits' DataFrame
    else: #Calculates specific limits per each position for fbx and fby
        new_order = [0, 3, 1, 4, 2, 5]
        ordered_means = [means[i] for i in new_order] #Reorder de the means for implementation
        for _ in range(int(measurements.shape[0] / (lenses_per_nest * 2))):  # Iterates over every nest (e.g. 24/6=4 nests)
            for j in range(len(ordered_means)):
                if j % 2 == 0:
                    low_limit = round(ordered_means[j] - x_tolerance, 4)
                    high_limit = round(ordered_means[j] + x_tolerance, 4)
                else:
                    low_limit = round(ordered_means[j] - y_tolerance, 4)
                    high_limit = round(ordered_means[j] + y_tolerance, 4)
                current_limits_df = pd.DataFrame({"LO_LIMIT": [low_limit], "HI_LIMIT": [high_limit]}) 
                limits = pd.concat([limits, current_limits_df], ignore_index=True, axis=0) 
    return limits

def ini_generator_personalized(limits: pd.DataFrame) -> None:
    '''Generates a ini file with personalized limits for every mean'''
    class CaseSensitiveConfigParser(configparser.ConfigParser):
        '''A custom class to override optionxform and avoid uppercases being converted to lowercase
        It just works F76 F76 F76 F76 F76'''
        def optionxform(self, optionstr):
            return optionstr
    config = CaseSensitiveConfigParser()
    config.read('../data/template.ini') #Import a template
    keys_list = []
    for section_name in config.sections(): #Get a keys list with the correct uppercased keys
        section = config[section_name]
        keys_list.extend(section.keys())
    HI_LIMIT = limits.iloc[:, 1]
    LO_LIMIT = limits.iloc[:, 0]
    for section in config.sections(): #Iterate through the sections and options in the .ini file
        keys_list = list(config[section].keys())
        j = 0
        for i in range(0, len(keys_list), 2):
            key1 = keys_list[i]
            key2 = keys_list[i + 1]
            col1 = str(limits.iloc[j, 1])
            col2 = str(limits.iloc[j, 0])
            j += 1
            config[section][key1] = col1
            config[section][key2] = col2
    for section in config.sections(): #Print the five first elements of the .ini for a quick check
        print(f"[{section}]")
        i = 0
        for key, value in config.items(section): 
            if i < 5:
                print(f"{key} = {value}")
                i += 1
            else:
                break
        print("...")
    #Save the modified data to a new .ini file
    with open(f'../a2_output/{glob.tooling}.ini', 'w') as configfile:
        for section in config.sections():
            configfile.write(f"[{section}]\n")
            keys = keys_list #Recover the original keys to write them in the .ini file
            for i, key in enumerate(keys):
                configfile.write(f"{key} = {config[section][key]}\n")
                if (i + 1) % 4 == 0 and i < len(keys) - 1: #Insert a blank line every four keys
                    configfile.write("\n")

In [3]:
#Data preparation
dbh = db.SQLite_Data_Extractor("database.db") #Connect to the database
measurements = dbh.retrieve(glob.tooling) #Get the desired tooling data
dbh.close_conn() 

Database *database.db* found in: c:\Users\luciano.galan\Desktop\Code\Python_Eiit_RyR\a2_RyR_Analyser\database\database.db
Table *TOP_Passat_B9* retrieved succesfully.
Closed connection to: c:\Users\luciano.galan\Desktop\Code\Python_Eiit_RyR\a2_RyR_Analyser\database\database.db


In [4]:
#Calculation of the absolute means for fiber x and fiber y
#limits = mean_calculator(measures) #Global means version
means = mean_calculator(measurements, glob.lenses_per_nest)
print(len(means))

Means per position (from lower to higher):
  Fiber x: 
[0.3305, 0.3392, 0.3324]
  Fiber y: 
[0.3328, 0.3312, 0.3393]
6


In [5]:
#Calculation of limits dataframe
#limits = limits_gen(measures, means) #Global means version
limits = limits_gen(measurements, means, glob.lenses_per_nest)
limits

,LO_LIMIT,HI_LIMIT
0,0.3180,0.3430
1,0.3178,0.3478
2,0.3267,0.3517
3,0.3162,0.3462
4,0.3199,0.3449
5,0.3243,0.3543
6,0.3180,0.3430
7,0.3178,0.3478
8,0.3267,0.3517
9,0.3162,0.3462


In [8]:
##Creation of the target dataframe with correct limits
new_column_names = [f'{i+1}' for i in range(len(measurements.columns))] #Columns renaming
measurements.columns = new_column_names
new_row_names = []; counter = 1; fiber = "X" #Preallocation
for i in range(0, len(measurements)): #Rows renaming
    new_row_name = f'Guia_Luz_Blanco_FB{counter}_{fiber}' 
    new_row_names.append(new_row_name)
    if i % 2 == 1: #Alternate the fiber axis and update the fober number every two fibers
        counter += 1
        fiber = "X"
    else:
        fiber = "Y"
measurements.index = new_row_names
limits.index = new_row_names
target = pd.concat([measurements, limits], axis=1) #Concatenates the measures and limits
target

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,LO_LIMIT,HI_LIMIT
Guia_Luz_Blanco_FB1_X,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,...,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,0.3181,0.3117,0.3367
Guia_Luz_Blanco_FB1_Y,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,...,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,0.3515,0.3176,0.3476
Guia_Luz_Blanco_FB2_X,0.3368,0.3402,0.3383,0.3325,0.3359,0.3372,0.3383,0.3356,0.3379,0.3406,...,0.3387,0.3394,0.3309,0.3390,0.3406,0.3375,0.3368,0.3402,0.3331,0.3581
Guia_Luz_Blanco_FB2_Y,0.3614,0.3631,0.3622,0.3593,0.3610,0.3616,0.3622,0.3609,0.3620,0.3633,...,0.3624,0.3627,0.3585,0.3625,0.3633,0.3618,0.3614,0.3631,0.3112,0.3412
Guia_Luz_Blanco_FB3_X,0.3465,0.3464,0.3466,0.3469,0.3469,0.3468,0.3467,0.3468,0.3465,0.3465,...,0.3467,0.3465,0.3484,0.3468,0.3466,0.3472,0.3472,0.3468,0.3188,0.3438
Guia_Luz_Blanco_FB3_Y,0.3708,0.3708,0.3707,0.3708,0.3708,0.3707,0.3707,0.3707,0.3707,0.3708,...,0.3707,0.3707,0.3709,0.3707,0.3707,0.3708,0.3707,0.3707,0.3301,0.3601
Guia_Luz_Blanco_FB4_X,0.3245,0.3245,0.3246,0.3248,0.3248,0.3248,0.3246,0.3248,0.3247,0.3246,...,0.3247,0.3246,0.3253,0.3248,0.3247,0.3250,0.3249,0.3248,0.3117,0.3367
Guia_Luz_Blanco_FB4_Y,0.3519,0.3519,0.3519,0.3519,0.3518,0.3519,0.3519,0.3518,0.3518,0.3519,...,0.3518,0.3519,0.3518,0.3519,0.3518,0.3519,0.3519,0.3518,0.3176,0.3476
Guia_Luz_Blanco_FB5_X,0.3345,0.3339,0.3342,0.3348,0.3342,0.3345,0.3339,0.3342,0.3337,0.3338,...,0.3337,0.3336,0.3356,0.3337,0.3337,0.3344,0.3342,0.3342,0.3331,0.3581
Guia_Luz_Blanco_FB5_Y,0.3587,0.3581,0.3585,0.3591,0.3584,0.3587,0.3581,0.3585,0.3578,0.3580,...,0.3578,0.3577,0.3600,0.3578,0.3578,0.3586,0.3585,0.3584,0.3112,0.3412


In [14]:
##Data export
confirmation = input("Do you want to export the data to a new Target.xlsx file? (y/n): ").strip().lower()
if confirmation == 'y':
    output_folder = os.path.abspath("../a2_output/Target_with_limits.xlsx")
    target.to_excel(output_folder, index=True, header=True)
    #os.startfile('../a2_output/Target_with_limits.xlsx')
    print(f'Data exported inside: {output_folder}')
else:
    print("Operation canceled.")

Operation canceled.


In [15]:
##Update of the database table
confirmation = input("Do you want to export the new limits to your database (y/n): ").strip().lower()
if confirmation == 'y':
    prepare_database(limits, glob.tooling+"_limits") #Store a df inside the database of the project
else:
    print("Operation canceled.")

Operation canceled.


In [16]:
###Test
resume = measurements.transpose().describe() #Transpose the df first due to describe() working in columns.
rough_means = list(resume.iloc[1, :].values)
print(len(rough_means))

24
